In [ ]:
import funcx
import sys
import psij

print("Python version : ", sys.version)
print("FuncX version : ", funcx.__version__)
print("Psij version: ", psij.__version__)

In [ ]:
theta_ep = "21d84df5-7eed-4b95-a4de-06b9c2b5a3d0"
midway_ep = "08e00632-4337-443d-822f-0948992bf0d3"
bebop_ep = "50906a25-f987-480c-a0dd-6e187c0d0f23"

In [ ]:
fx_theta = funcx.FuncXExecutor(endpoint_id=theta_ep)
fx_midway = funcx.FuncXExecutor(endpoint_id=midway_ep)
fx_bebop = funcx.FuncXExecutor(endpoint_id=bebop_ep)

In [ ]:
def platinfo():
    import platform
    return platform.uname()

In [ ]:
fu_theta = fx_theta.submit(platinfo)

In [ ]:
fu_theta.result()

In [ ]:
fu_midway = fx_midway.submit(platinfo)

In [ ]:
fu_midway.result()

In [ ]:
fu_bebop = fx_bebop.submit(platinfo)

In [ ]:
fu_bebop.result()

In [ ]:
def submit_job():
    import logging
    logging.basicConfig(filename='', encoding='utf-8', level=logging.DEBUG)
    from psij import Job, JobExecutor, JobSpec, JobAttributes, ResourceSpecV1
    
    executor = JobExecutor.get_instance("slurm")

    job = Job(
        JobSpec(
            executable="/bin/sleep",
            arguments=["60"],
            resources=ResourceSpecV1(node_count=1),
            attributes=JobAttributes(
                queue_name="broadwl",
                project_name="pi-chard",
            ),
        )
    )

    executor.submit(job)
    return job.native_id

In [ ]:
job_fu = fx.submit(submit_job, endpoint_id=midway_ep)

In [ ]:
print(job_fu.result())
status_id = job_fu.result()

In [ ]:
def status_job(native_id):
    from psij import Job, JobExecutor
    import time
    
    executor = JobExecutor.get_instance("slurm")
    job = Job()
    executor.attach(job, native_id)
    time.sleep(5)
    return job.status

In [ ]:
status_fu = fx.submit(status_job, status_id, endpoint_id=midway_ep)

In [ ]:
x = status_fu.result()

In [ ]:
str(x)

In [ ]:
def submit_job_cobalt():
    from psij import Job, JobExecutor, JobSpec, JobAttributes, ResourceSpecV1
    from psij import JobExecutorConfig
    
    executor = JobExecutor.get_instance("cobalt")

    job = Job(
        JobSpec(
            executable="/bin/sleep",
            arguments=["60"],
            resources=ResourceSpecV1(node_count=1),
            attributes=JobAttributes(
                project_name='CSC249ADTR01',
                queue_name='debug-cache-quad'
            ),
        )
    )

    executor.submit(job)
    
    return job.native_id

In [ ]:
job_fu = fx_theta.submit(submit_job_cobalt)

In [ ]:
job_id = job_fu.result()
print("Job_id : ", job_id)

In [ ]:
def status_job(native_id):
    from psij import Job, JobExecutor
    import time
    
    executor = JobExecutor.get_instance("cobalt")
    job = Job()
    executor.attach(job, native_id)
    time.sleep(3)
    return job.status

In [ ]:
status_fu = fx_theta.submit(status_job, job_id)

s = status_fu.result()

In [ ]:
print(s)

In [ ]:
def cancel_job(native_id, scheduler="cobalt"):
    from psij import Job, JobExecutor
    import time
    
    executor = JobExecutor.get_instance(scheduler)
    job = Job()
    executor.attach(job, native_id)
    time.sleep(3)
    return job.cancel()

In [ ]:
cancel_fu = fx_theta.submit(cancel_job, job_id)

s = cancel_fu.result()
print(s)

In [ ]:
def submit_job_bebop():
    import logging
    logging.basicConfig(filename='', encoding='utf-8', level=logging.DEBUG)
    from psij import Job, JobExecutor, JobSpec, JobAttributes, ResourceSpecV1
    
    executor = JobExecutor.get_instance("slurm")

    job = Job(
        JobSpec(
            executable="/bin/sleep",
            arguments=["60"],
            resources=ResourceSpecV1(node_count=1),
            attributes=JobAttributes(
                queue_name="bdws",
                project_name="EMEWS",
            ),
        )
    )

    executor.submit(job)
    return job.native_id

In [ ]:
future = fx_bebop.submit(submit_job_bebop)

In [ ]:
job_id = future.result()
print(f"Job id on bebop: {job_id}")

In [ ]:
future = fx_bebop.submit(status_job, job_id)

In [ ]:
status = future.result()
print(status)

In [ ]:
future = fx_bebop.submit(cancel_job, job_id, scheduler="slurm")

In [ ]:
print("Output from cancel:", future.result())